In [ ]:
import xarray as xr
import numpy as np
import hydromt
import geopandas as gpd
from os.path import join
from hydromt.models import SfincsModel
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import colors, patheffects
import cartopy.crs as ccrs

In [ ]:
mod = SfincsModel(r'../../3_models/SFINCS2/01_rivpowlaw/idai', mode='r')
ds_sfx = mod.staticmaps
gdf_points = mod.staticgeoms['obs']

dep_mask = ds_sfx['dep']==-9999
riv_mask = hydromt.open_raster(join(mod.root, '../gis', 'rivmsk.tif'))
riv_mask.raster.set_nodata(0)
gdf_riv = riv_mask.where(~dep_mask, 0).raster.vectorize()

In [ ]:
root = r'../../3_models/SFINCS2/00_base_100m'
mod0 = SfincsModel(root, mode='r')
flwdir =  hydromt.flw.flwdir_from_da(mod0.staticmaps['flwdir'])
xs, ys = flwdir.xy(flwdir.idxs_pit)
selection = ~np.logical_and(xs>=680000, ys>=7820000)
mod0.set_staticmaps(flwdir.basins(xy=(xs[selection], ys[selection])).astype('int16'), 'basins')
mod0.staticmaps['basins'].raster.set_nodata(0)
da_msk_bas = np.logical_and(mod0.staticmaps['basins'] == 0, mod0.staticmaps['dep']>0)
da_msk_bas = da_msk_bas.astype('int16')
da_msk_bas.raster.set_nodata(0)
gdf_bas = da_msk_bas.raster.vectorize()
# gdf_bas.plot()

In [ ]:
from hydromt_sfincs.workflows import river_boundary_points
gdf_src0, gdf_riv0 = river_boundary_points(
    da_flwdir=mod0.staticmaps['flwdir'],
    da_uparea=mod0.staticmaps['uparea'].where(da_msk_bas==0, 0),
    river_upa=10,
    river_len=10000,
)

gdf_src0.sort_values(by='uparea', ascending=False).iloc[mod0.forcing['dis'].index.size:,]#.sum()

In [ ]:
gdf0 = gpd.read_file(join(r'../../1_data/4_observations', 'gauges.gpkg')).set_index('name').to_crs(mod.crs)
gdf0

In [ ]:
#towns
df = pd.DataFrame({
    'Mefambisse': (-19.53935304519779, 34.629741259143714),
    'Buzi': (-19.883677, 34.599003),        
    'Beira': (-19.825007644673512, 34.86483377443984),    
}).T
df.columns = ['y', 'x']
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']), crs=4326).to_crs(mod.crs)
gdf.to_file(r'../../1_data/towns.geojson', driver='GeoJSON')
gdf.head()

In [ ]:
gdf_bnd = mod.forcing['bzs'].vector.to_gdf()
gdf_src = mod.forcing['dis'].vector.to_gdf()
gdf_region = mod.region

In [ ]:
cmf_root = r'../../3_models/CMF/03min/gis/'
ds_cmf = hydromt.open_mfraster(join(cmf_root, r'*.tif'))
ds_cmf.raster.set_crs(4326)
gdf_gtsm = gpd.read_file(join(cmf_root, 'gtsm.geojson'))
basins = [1,2,23]

In [ ]:
gdf_cmf_riv = gpd.read_file(join(cmf_root, 'nextxy_outlets.geojson'))
gdf_cmf_riv['uparea10'] = np.log10(np.maximum(gdf_cmf_riv['uparea'].values, 1))
gdf_cmf_riv.head()
gdf_cmf_riv = gdf_cmf_riv[np.isin(gdf_cmf_riv['basin'], basins)]
gdf_cmf_riv = gdf_cmf_riv.to_crs(ds_sfx.raster.crs)
gdf_cmf_riv

In [ ]:
# colrows = [(25,30), (27,30), (26, 25), (26,26), (28,28)]
# cols, rows = zip(*colrows)
# cols = xr.IndexVariable('index', np.asarray(cols)-1)
# rows = xr.IndexVariable('index', np.asarray(rows)-1)
# xs, ys = ds_cmf.isel(x=cols, y=rows)['lonlat'].values
# gdf_pnts = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=xs, y=ys), crs=4326).to_crs(ds_sfx.raster.crs)
# gdf_pnts.index = gdf_pnts.index+1
# gdf_pnts.to_file('obs_locs.geojson', driver='GeoJSON')

In [ ]:
deltares_data=True

if deltares_data:
    # basin shapes
    cat = hydromt.DataCatalog(deltares_data=deltares_data)
    da_bas = cat.get_rasterdataset('merit_hydro', variables=['basins'], bbox=ds_cmf.raster.bounds, buffer=10)
    bas_lst = np.unique(da_bas)[4:6]
    print(bas_lst)

    # hillshade background
    da = cat.get_rasterdataset('merit_hydro', variables=['elevtn'], bbox=ds_cmf.raster.bounds, buffer=10)
    res = 500
    da = da.raster.reproject(dst_crs=ds_sfx.raster.crs, dst_res=res, method='cubic')


else:
    da_bas = ds_cmf['basin']
    da= ds_cmf['elevtn']
    res = 5000
    bas_lst = [1,2]#,23]


ls = colors.LightSource(azdeg=210, altdeg=45)
hs = ls.hillshade(np.ma.masked_equal(da.values, -9999), vert_exag=10, dx=res, dy=res)
da_hs = xr.DataArray(
    dims=da.raster.dims, data=hs, coords=da.raster.coords
).where(~hs.mask)

gdf_cmf_bas = da_bas.astype(np.int32).raster.vectorize()
gdf_cmf_bas['basin'] = gdf_cmf_bas['value'].astype(int)
gdf_cmf_bas = gdf_cmf_bas[np.isin(gdf_cmf_bas['basin'], bas_lst)].set_index('basin')
gdf_cmf_bas = gdf_cmf_bas.to_crs(ds_sfx.raster.crs)

In [ ]:
vmin, vmax = (
    ds_sfx["dep"].raster.mask_nodata().quantile([0.0, 0.98]).values
)
c_bat = plt.cm.terrain(np.linspace(0, 0.17, 256))
c_dem = plt.cm.terrain(np.linspace(0.25, 0.9, 256))
c_all = np.vstack((c_bat, c_dem))
cmap = colors.LinearSegmentedColormap.from_list("bat_dem", c_all)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
da_dep = ds_sfx['dep'].where(ds_sfx['dep']!=ds_sfx['dep'].raster.nodata)
da_dep.attrs.update(unit='m+EGM96')

In [ ]:
import matplotlib.patches as mpatches
from matplotlib_scalebar.scalebar import ScaleBar

# read crs and utm zone > convert to cartopy
wkt = ds_sfx.raster.crs.to_wkt()
if "UTM zone " not in wkt:
    raise ValueError("Model CRS UTM zone not found.")
utm_zone = ds_sfx.raster.crs.to_wkt().split("UTM zone ")[1][:3]
utm = ccrs.UTM(int(utm_zone[:2]), "S" in utm_zone)

ann_kwargs = dict(
    xytext=(4, 0),
    textcoords="offset points",
    zorder=4,
    path_effects=[
        patheffects.Stroke(linewidth=3, foreground="w"),
        patheffects.Normal(),
    ],
)

fig, (ax, ax1) = plt.subplots(
    1, 2, figsize=(16,10),
    subplot_kw={'projection': utm},
)


## CAMA-FLOOD ##
extent = np.array(gdf_cmf_bas.total_bounds)[[0, 2, 1, 3]]
ax.set_extent(extent, crs=utm)

da_hs.plot(ax=ax, cmap='Greys', add_colorbar=False, alpha=0.4, zorder=0)
# basins
kwargs = dict(color='orangered', alpha=0.4, label='Buzi basin')
gdf_cmf_bas.iloc[[1]].plot(ax=ax, zorder=1, **kwargs)
buzi_handle = mpatches.Patch(**kwargs)
kwargs = dict(color='darkgreen', alpha=0.4, label='Pungwe basin')
gdf_cmf_bas.iloc[[0]].plot(ax=ax, zorder=1, **kwargs)
pungwe_handle = mpatches.Patch(**kwargs)
# river
lws = (gdf_cmf_riv['uparea10'])/1.5
gdf_cmf_riv.plot(ax=ax, zorder=1, label='CaMa-Flood rivers', lw=lws)
# gauges
gdf0.plot(ax=ax, marker='o', markersize=75, c="w", edgecolor="k", label='river gauge\n approx. locations', zorder=4)
for label, grow in gdf0.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)
# gtsm
gdf_gtsm.plot(ax=ax, marker="^", markersize=75, c="w", edgecolor="k", label='waterlevel bound (GTSM)', zorder=4)
# sfincs region
kwargs = dict(facecolor='none', edgecolor='k', lw=1, label='nested SFINCS model')
gdf_region.plot(ax=ax, zorder=3, **kwargs)
sfincs_handle = mpatches.Patch(**kwargs)

ax.yaxis.set_visible(True)
ax.xaxis.set_visible(True)
ax.set_ylabel(f"y coordinate UTM zone {utm_zone} [m]")
ax.set_xlabel(f"x coordinate UTM zone {utm_zone} [m]")

handles, _ = ax.get_legend_handles_labels()
ax.legend(
    handles=[pungwe_handle, buzi_handle, *handles, sfincs_handle],
    bbox_to_anchor=(1, 1),
    title="Legend",
    loc="upper right",
    frameon=True,
)
ax.add_artist(ScaleBar(1, location='lower right', box_alpha=0.0))
ax.set_title('A) CaMa-Flood model - rivers')

## SFINCS ##

extent = np.array(gdf_region.buffer(1000).total_bounds)[[0, 2, 1, 3]]
ax1.set_extent(extent, crs=utm)
cs = da_dep.plot(ax=ax1, cmap=cmap, norm=norm, zorder=2, add_colorbar=True, cbar_kwargs=dict(shrink=0.5, anchor=(0,0.1)))
gdf_bnd.plot(ax=ax1, marker="^", markersize=60, c="w", edgecolor="k", label='waterlevel boundary', zorder=4)
for label, grow in gdf_bnd.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'H{label}', xy=(x, y), **ann_kwargs)
gdf_src.plot(ax=ax1, marker=">", markersize=60, c="w", edgecolor="k", label='discharge boundary', zorder=4)
for label, grow in gdf_src.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'Q{label}', xy=(x, y), **ann_kwargs)
gdf_points.plot(ax=ax1, marker='d', markersize=60, c="w", edgecolor="k", label='model point', zorder=4)
for label, grow in gdf_points.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'{label}', xy=(x, y), **ann_kwargs)
ax1.text(0.7, 0.08, 'Mozambique Channel\n(Indian Ocean)', rotation=35, transform=ax1.transAxes)
gdf_riv.boundary.plot(ax=ax1, ls='--', lw=0.5, color='k', alpha=0.5, label='river outline')

kwargs = dict(facecolor='white', alpha=0.6, lw=0, hatch='x', label='excluded area')
gdf_bas.plot(ax=ax1, zorder=3, **kwargs)
bas_handle = mpatches.Patch(**kwargs)

# towns
gdf.plot(ax=ax1, marker='.', markersize=40, color="k", label='towns / cities', zorder=4)
for label, grow in gdf.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'{label}', xy=(x, y), **ann_kwargs)
    
# # Add a colorbar axis at the bottom of the graph
# cbar_ax = fig.add_axes([0.91, 0.14, 0.015, 0.3])
# # # Draw the colorbar
# cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
# cbar_ax.set_ylabel('elevation [m+EGM96]')

ax1.yaxis.set_visible(True)
ax1.xaxis.set_visible(True)
ax1.set_ylabel(f"")
ax1.set_xlabel(f"x coordinate UTM zone {utm_zone} [m]")
handles, _ = ax1.get_legend_handles_labels()
ax1.legend(
    handles=[*handles, bas_handle],
    bbox_to_anchor=(1.0, 1),
    title="Legend",
    loc="upper right",
    frameon=True,
    framealpha=0.9,
)
ax1.add_artist(ScaleBar(1, location='lower right', box_alpha=0.0))
ax1.set_title('B) SFINCS model - elevation')

#
fig.subplots_adjust(wspace=0.04)
plt.savefig(join('../../4_results/rebuttal', f'nested_spatial.png'), dpi=225, bbox_inches="tight")